# Data Smart Sans Excel

< need to add linker to master >

In [1]:
# Download and extract Data Smart files
import os
import urllib.request
import zipfile
if not os.path.exists(os.path.join(os.getcwd(), "data_smart_files")):
    r = range(1, 11)
    url_list = []
    url_base = "http://media.wiley.com/product_ancillary/6X/11186614/DOWNLOAD/"
    for i in r:
        url_list.append(url_base + 'ch' + str(i).zfill(2) + '.zip')
    for url in url_list:
        file_name = url[-8:]
        urllib.request.urlretrieve (url, file_name)
        z = zipfile.ZipFile(file_name, "r")
        z.extractall(os.path.join(os.getcwd(), "data_smart_files"))
        z.close()

## Chapter 3 - Naïve Bayes

In [16]:
import pandas as pd
import numpy as np
excel_file = os.path.join(os.getcwd(), "data_smart_files", "", "Mandrill.xlsx")
about_mandrill_df = pd.read_excel(excel_file, 'AboutMandrillApp', parse_cols = "A", index_col=None)
about_other_df = pd.read_excel(excel_file, 'AboutOther', parse_cols = "A", index_col=None)

def clean(s): 
    s = s.lower()
    s = s.replace('. ', ' ')
    s = s.replace(': ', ' ')
    s = s.replace('?', ' ')
    s = s.replace('!', ' ')
    s = s.replace(';', ' ')
    s = s.replace(',', ' ')
    return s

def split_(s):
    return s.split(' ')

about_mandrill_df = about_mandrill_df.applymap(clean)
about_mandrill_df = about_mandrill_df.applymap(split_)
    
about_other_df = about_other_df.applymap(clean)
about_other_df = about_other_df.applymap(split_)

In [25]:
import itertools
about_mandrill_words = []
for row in about_mandrill_df.itertuples():
    about_mandrill_words.append(row[1])
about_mandrill_words  = list(itertools.chain(*about_mandrill_words))

about_other_words = []
for row in about_other_df.itertuples():
    about_other_words.append(row[1])
about_other_words  = list(itertools.chain(*about_other_words))

In [27]:
about_mandrill_count_dict = {}
for word in set(about_mandrill_words):
    about_mandrill_count_dict[word] = about_mandrill_words.count(word)
    
about_other_count_dict = {}
for word in set(about_other_words):
    about_other_count_dict[word] = about_other_words.count(word)